In [5]:
## Run selenium and chrome driver to scrape data from cloudbytes.dev
import time
import os.path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [6]:
## Setup chrome options
chrome_options = Options()
chrome_options.headless = True # Ensure GUI is off
# chrome_options.add_argument("--window-size=1920,1200")

In [7]:
# Set path to chromedriver as per your configuration
homedir = os.path.expanduser("~")
webdriver_service = Service(f"{homedir}/ao3lockwood-co/chromedriver")

In [8]:
# Choose Chrome Browser
browser = webdriver.Chrome(service=webdriver_service, options=chrome_options)

In [9]:
# Get page
pagenum=1
link="https://archiveofourown.org/tags/Lockwood%20*a*%20Co*d*%20-%20Jonathan%20Stroud/works?page="+str(pagenum)
#link="https://archiveofourown.org/tags/Lockwood%20*a*%20Co*d*%20-%20Jonathan%20Stroud/works?page=1"
browser.get(link)

maxpagenum=int(browser.find_element(By.XPATH,'//ol[1]/li[13]').text.strip())

In [10]:
def get_links(browser):
    works = browser.find_elements(By.XPATH, '//ol[2]/li')
    # Iterate through each work and extract author and datetime
    data=[]
    for work in works:
        h4 = work.find_element(By.TAG_NAME,'h4')
        a = h4.find_elements(By.TAG_NAME, 'a')
        links = a[0].get_attribute("href")
        data.append(links)
    return data
    

In [11]:
# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d%m%Y_%H%M")
print("date and time =", dt_string)

# Create an empty DataFrame to hold the data
data_list = get_links(browser)
print('page 1 has been processed')
# Iterate through each page and append the data to the DataFrame
for p in range(2,maxpagenum+1):
    pagenum=p
    time.sleep(10)
    print(f'procesing page {pagenum}/{maxpagenum}')
    link="https://archiveofourown.org/tags/Lockwood%20*a*%20Co*d*%20-%20Jonathan%20Stroud/works?page="+str(pagenum)
    browser.get(link)
    data_list=data_list + (get_links(browser))
    print(len(data_list))

#Wait for 10 seconds
time.sleep(10)
browser.quit()

now = 2023-04-27 11:26:11.782821
date and time = 27042023_1126
page 1 has been processed
procesing page 2/69
40
procesing page 3/69
60
procesing page 4/69
80
procesing page 5/69
100
procesing page 6/69
120
procesing page 7/69
140
procesing page 8/69
160
procesing page 9/69
180
procesing page 10/69
200
procesing page 11/69
220
procesing page 12/69
240
procesing page 13/69
260
procesing page 14/69
280
procesing page 15/69
300
procesing page 16/69
320
procesing page 17/69
340
procesing page 18/69
360
procesing page 19/69
380
procesing page 20/69
400
procesing page 21/69
420
procesing page 22/69
440
procesing page 23/69
460
procesing page 24/69
480
procesing page 25/69
500
procesing page 26/69
520
procesing page 27/69
540
procesing page 28/69
560
procesing page 29/69
580
procesing page 30/69
600
procesing page 31/69
620
procesing page 32/69
640
procesing page 33/69
660
procesing page 34/69
680
procesing page 35/69
700
procesing page 36/69
720
procesing page 37/69
740
procesing page 38/69
7

In [12]:
data_list

['https://archiveofourown.org/works/46445644',
 'https://archiveofourown.org/works/46466728',
 'https://archiveofourown.org/works/46188124',
 'https://archiveofourown.org/works/45793852',
 'https://archiveofourown.org/works/45359485',
 'https://archiveofourown.org/works/46775158',
 'https://archiveofourown.org/works/45376639',
 'https://archiveofourown.org/works/46773547',
 'https://archiveofourown.org/works/45361381',
 'https://archiveofourown.org/works/46772158',
 'https://archiveofourown.org/works/46771864',
 'https://archiveofourown.org/works/46616701',
 'https://archiveofourown.org/works/46519282',
 'https://archiveofourown.org/works/46769455',
 'https://archiveofourown.org/works/46757398',
 'https://archiveofourown.org/works/45992374',
 'https://archiveofourown.org/works/46680625',
 'https://archiveofourown.org/works/46754731',
 'https://archiveofourown.org/works/46315132',
 'https://archiveofourown.org/works/46760389',
 'https://archiveofourown.org/works/46464568',
 'https://arc

In [13]:
def get_data(links_list):
    data=[]
    counter=0
    for x in links_list:
        newlink=x+'?view_adult=true'
        counter+=1
        print(f"getting missing data {counter}/{len(links_list)}")
        source = requests.get(newlink, headers={
                          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)'}).text
        soup = BeautifulSoup(source,'html.parser')
        title=soup.find('h2', attrs={'class':'title heading'}).get_text().replace('\n','').strip()
        try:
            author=soup.find('a', attrs={'rel':'author'}).get_text()
        except:
            author="Anonymous"
        datetime=soup.find('dd', attrs={'class':'published'}).get_text()
        try:
            updated = soup.find('dd', attrs={'class':'status'}).get_text()
        except:
            updated=datetime
        chapters=soup.find('dd', attrs={'class':'chapters'}).get_text()
        language=soup.find('dd', attrs={'class':'language'}).get_text().replace('\n','').strip()
        words=soup.find('dd', attrs={'class':'words'}).get_text()
        try:
            kudos=soup.find('dd', attrs={'class':'kudos'}).get_text()
        except:
            kudos=0
        try:
            comments=soup.find('dd', attrs={'class':'comments'}).get_text()
        except:
            comments=0
        try:
            bookmarks=soup.find('dd', attrs={'class':'bookmarks'}).get_text()
        except:
            bookmarks=0
        try:
            hits=soup.find('dd', attrs={'class':'hits'}).get_text()
        except:
            hits=0
        warning=soup.find('dd', attrs={'class':'warning tags'}).get_text().replace('\n','').strip()
        try:
            ships = soup.find('dd', attrs={'class':'relationship tags'})
            ships_list=ships.find_all('a', attrs={'class':'tag'})
            mainrelationship = ships_list[0].get_text().strip()
            relationship_list = []
            for r in ships_list:
                relationship_list.append(r.get_text().strip())
        except:
            mainrelationship='None'
            relationship_list = []
        try:
            char = soup.find('dd', attrs={'class':'character tags'})
            char_list=char.find_all('a', attrs={'class':'tag'})
            character_list = []
            for c in char_list:
                character_list.append(c.get_text().strip())
        except:
            character_list=[]
        try:
            freeform = soup.find('dd', attrs={'class':'freeform tags'})
            freefom_list=freeform.find_all('a', attrs={'class':'tag'})
            tags_list = []
            for t in freefom_list:
                tags_list.append(t.get_text().strip())
        except:
            tags_list = []
        try:
            position = soup.find('span', attrs={'class':'position'})
            position_list = position.find('a')
            series_list = []
            for p in position_list:
                series_list.append(p.get_text().strip())
        except:
            series_list = 'not a series'
        try:
            summary=soup.find('div', attrs={'class':'summary module'}).get_text().replace('\n', ' ').replace('Summary:','').strip()
        except:
            summary=np.nan
        rating_tag=soup.find('dd', attrs={'class':'rating tags'}).get_text().replace('\n','').strip()
        row = {'link':x, 
               'title':title,
               'author':author,
               'published':datetime, 
               'updatedate':updated,
               'chapters':chapters,
               'language':language,
               'words':words,
               'kudos':kudos, 
               'comments':comments,
               'bookmarks':bookmarks,
               'hits':hits,
               'warning':warning,
               'mainship':mainrelationship,
               'relationship':relationship_list,
               'characters':character_list, 
               'tags':tags_list,
               'summary':summary,
               'rating':rating_tag
               }
        #print(row)
        data.append(row)
        time.sleep(10)
    return pd.DataFrame(data)

In [14]:
final=get_data(data_list)
# Split the chapter column into chapter and chapter_max, and create a completion column
final[['chapter','chapter_max']] = final.chapters.str.split("/", expand=True)
final['completion'] = final.apply(lambda row: 'completed' if row['chapter']==row['chapter_max'] else 'incomplete', axis=1)


getting missing data 1/1361
getting missing data 2/1361
getting missing data 3/1361
getting missing data 4/1361
getting missing data 5/1361
getting missing data 6/1361
getting missing data 7/1361
getting missing data 8/1361
getting missing data 9/1361
getting missing data 10/1361
getting missing data 11/1361
getting missing data 12/1361
getting missing data 13/1361
getting missing data 14/1361
getting missing data 15/1361
getting missing data 16/1361
getting missing data 17/1361
getting missing data 18/1361
getting missing data 19/1361
getting missing data 20/1361
getting missing data 21/1361
getting missing data 22/1361
getting missing data 23/1361
getting missing data 24/1361
getting missing data 25/1361
getting missing data 26/1361
getting missing data 27/1361
getting missing data 28/1361
getting missing data 29/1361
getting missing data 30/1361
getting missing data 31/1361
getting missing data 32/1361
getting missing data 33/1361
getting missing data 34/1361
getting missing data 35

In [15]:
filename=f'ao3_lockwood_and_co_ao_{dt_string}.csv'
final.to_csv(filename, index=False)

In [16]:
working_df = pd.read_csv(filename)

In [17]:
working_df['published'] = pd.to_datetime(working_df['published'])
working_df['updatedate'] = pd.to_datetime(working_df['updatedate'])
working_df['currentdate'] = max(working_df['updatedate'])
working_df['datediff_pub'] = (working_df['currentdate']-working_df['published'])/np.timedelta64(1,'D')
working_df['datediff'] = (working_df['currentdate']-working_df['updatedate'])/np.timedelta64(1,'D')

In [18]:
working_df['classification'] = working_df.apply(lambda row: 'oneshot' if row['chapter_max']=='1' else ('multichapter(complete)' if row['completion']=='completed' else ('multichapter(updating)' if row['datediff']<=60 else 'multichapter(dormant)')), axis=1)


In [ ]:
working_df['pubupdate'] = working_df.apply(lambda row: 'published' if row['datediff_pub']<=7 else ('updated' if row['datediff']<=7 else 'na'))

In [19]:
def get_unique_item(column):
    item_dic={}
    for row in column:
        row_item = row.replace("[","").replace("]","").replace("'","").replace('"','').split(",")
        for item in row_item:
            item=item.strip()
            if item in item_dic.keys():
                item_dic[item] = item_dic[item]+1
            else:
                item_dic[item] = 1
    item_keys=list(item_dic.keys())
    item_keys.sort()
    sorted_item = {i: item_dic[i] for i in item_keys}
    for k, v in sorted_item.items():
        print(k, v)
    return sorted_item

In [20]:
def get_num_item(column):
    item=[]
    for row in column:
        row_item = row.replace("[","").replace("]","").replace("'","").replace('"','').split(",")
        if row_item!=['']:
            item.append(len(row_item))
        else:
            item.append(0) 
    return item

In [21]:
character=get_unique_item(working_df['characters'])

 47
(Briefly) George Cubbins 1
(Briefly) Holly Munro 1
(Briefly) Quill Kipps 1
(Cameo) 1
(Mentioned) Bubs 1
(Mentioned) Celia Lockwood 1
(Mentioned) Donald Lockwood 1
(Mentioned) Hera Syndulla 1
(Mentioned) Holly Munro 1
(Mentioned) Jessica Lockwood 1
(also just mentioned) 1
(also mentioned) 1
(barely) - Character 1
(briefly) 1
(implied) - Character 1
(mentioned) 3
(the last three are only mentioned) 1
35 Portland Row - Character 2
Aaron Hotchner 1
Aaron Minyard 1
Abigail Hobbs 1
Adelaide Winkman 3
Adora (She-Ra) 1
Adult Lockwood 1
Adult Lucy - Character 1
Aerith Gainsborough 1
Aikemere 1
Albert Browne 2
Albert Joplin 3
Albert Joplin | Pamela Joplin 5
Albus Dumbledore 1
Alcatraz Smedry 1
Alcina Dimitrescu 1
Alcina Dimitrescus Daughters 1
Alec Lightwood 2
Aleksander of Hohenburg 1
Alfie Morgan 2
Alfie Morgan (mentioned) 1
Alfred Pennyworth 1
Allan-a-Dale 1
Alma Peregrine 1
Amina El Maghrabi 1
Andrew Minyard 1
Angie (Resident Evil) 1
Annabel Ward 5
Annabelle Ward 1
Annabeth Chase (Percy 

In [22]:
relationship=get_unique_item(working_df['relationship'])

 114
(Background) Lucy Carlyle/Anthony Lockwood 2
(Briefly) Lucy Carlyle & Holly Munro 1
(IMPLIED) 1
(suggested) Bartimaeus/Ptolemy 1
Adora/Catra (She-Ra) 1
Aerith Gainsborough/Tifa Lockhart 1
Albert Browne & Scarlett McCain 1
Albert Browne/Scarlett McCain 2
Alcina Dimitrescu/Original Female Character(s) 1
Alcina Dimitrescu/Reader 1
Aleksander of Hohenberg/Deryn Sharp 1
Amina El Maghrabi/Daisy Wells 1
Annabel Ward/John Fairfax 1
Annabeth Chase/Percy Jackson 2
Anthony Bridgerton & Francesca Bridgerton 1
Anthony Bridgerton/Kate Sharma (mentioned) 1
Anthony Bridgerton/Kate Sheffield | Kate Sharma 6
Anthony Lockwood & Albert Brown 1
Anthony Lockwood & George Cubbins 2
Anthony Lockwood & George Karim 2
Anthony Lockwood & Holly Munro 20
Anthony Lockwood & Inspector Barnes 2
Anthony Lockwood & Jessica Lockwood 18
Anthony Lockwood & Julius Winkman 1
Anthony Lockwood & Lucy Carlyle 1
Anthony Lockwood & Lucy Carlyle & George Cubbins 1
Anthony Lockwood & Lucy Carlyle & George Karim | George Cubbi

In [23]:
tags=get_unique_item(working_df['tags'])

 38
#LetGeorgeSayFuck2023 1
(AFFECTIONATE) 1
(EVENTUALLY) resolved sexual tension 1
(LET ME FULFILL MY FANTASIES OKAY) 1
(Lucy Carlyle being Lockwoods favourite) 1
(Or is it?) 1
(Theyre all in their early 20s & Kipps is mid-20s) 1
(a teensy bit) 1
(and unsure how to tag this) 1
(as expected) 1
(at least in my canon) 1
(because Lockwood needs the comfort more than Lucy does) 1
(because of course I couldnt just stop at 5) 1
(but in a lighthearted way) 1
(dont shoot the messenger) 1
(eventually) - Freeform 1
(for reproductive purposes) 1
(he’s going to Area 51 don’t tell Lucy) 1
(how was that not already a tag) 1
(if there are spoilers later on they will be mild with warnings) 1
(if you dont count the skull) 1
(if you squint really hard) 1
(if youve ever thought that to yourself then youre in the right place) 1
(implied) - Freeform 1
(in the loosest possible definition) 1
(just a lil bit) 1
(looks at major character death tag in a hunger games au) 1
(loosely based regency im not doing eno

In [24]:
author_df = working_df.groupby(['author'], as_index=False).agg({'updatedate':'max', 'published':'min'})
author_df = author_df.rename(columns={'updatedate':'lastauthorupdate','published':'firstauthorupdate'})

In [25]:
if 'firstauthorupdate_x' in working_df.columns:
    working_df=working_df.drop(columns=['firstauthorupdate_x','lastauthorupdate_x', 'lastauthorupdate_y','firstauthorupdate_y'])
    working_df=working_df.merge(author_df, how='left', on='author')
else:
    working_df=working_df.merge(author_df, how='left', on='author')

In [26]:
working_df['author_lastupdate_diff'] = (working_df['currentdate']-working_df['lastauthorupdate'])/np.timedelta64(1,'D')
working_df['daysactive'] = (working_df['lastauthorupdate']-working_df['firstauthorupdate'])/np.timedelta64(1,'D')
working_df['daysincefirtupload'] = (working_df['currentdate']-working_df['firstauthorupdate'])/np.timedelta64(1,'D')
working_df['author_activity'] = working_df['author_lastupdate_diff'].apply(lambda x: 'active' if x<=60 else 'inactive')

In [ ]:
working_df['new_writer'] = working_df.apply(lambda row: 'new' if row['daysincefirtupload']<=7 else 'old')

In [27]:
working_df[['lastauthorupdate','firstauthorupdate','daysactive']].sort_values(by=['daysactive'], ascending=False)

,lastauthorupdate,firstauthorupdate,daysactive
1333,2023-02-04,2014-02-05,3286.0
872,2023-02-04,2014-02-05,3286.0
1294,2023-02-04,2014-02-05,3286.0
1348,2023-02-04,2014-02-05,3286.0
1295,2023-02-04,2014-02-05,3286.0
...,...,...,...
961,2022-08-05,2022-08-05,0.0
187,2023-04-10,2023-04-10,0.0
532,2023-03-14,2023-03-14,0.0
185,2023-04-10,2023-04-10,0.0


In [28]:
working_df['num_relationship']=get_num_item(working_df['relationship'])
working_df['num_characters']=get_num_item(working_df['characters'])
working_df['num_tags']=get_num_item(working_df['tags'])

In [29]:
working_df.to_csv(filename, index=False)